In [1]:
# Practica 13 - BORME
# María José Medina Hernández

In [2]:
# Imports
import requests
import time
import logging
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from os import makedirs
from os.path import exists

In [3]:
# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("info")

INFO:root:info


In [4]:
boletin = "20220105" #12/12/2021

In [14]:
# Create directory for the PDFs
directory = f"Boletin-{boletin}"
if not exists(directory):
    makedirs(directory)

In [6]:
def format_date(date):
    """
    Formats "YYYYMMDD" date into "DD/MM/YYYY" date.
    
    Params
    -------
    date : string
        The date string "YYYYMMDD".

    Return
    -------
    f_date : string
        Formatted date "DD/MM/YYYY".
    """
    date = date.strip()
    year = date[0:4]
    month = date[4:6]
    day = date[6:8]
    f_date = f"{day}/{month}/{year}"
    return f_date

In [7]:
date = format_date(boletin)

In [8]:
# Deploy the Chrome diver executed by Selenium
chromedriver_path = r"..\chromedriver_win32\chromedriver"
# Path to the Chrome driver
driver = webdriver.Chrome(executable_path=chromedriver_path)
url = "https://www.boe.es/diario_borme/"
driver.get(url)
time.sleep(2)

<ipython-input-8-9ea946ed2044>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chromedriver_path)


In [9]:
# Input the desired date in the calendar
driver.find_element_by_id("fechaBORME").send_keys(date)
divs = driver.find_element_by_name("acc").click()

<ipython-input-9-cb102da005e4>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("fechaBORME").send_keys(date)
<ipython-input-9-cb102da005e4>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  divs = driver.find_element_by_name("acc").click()


In [10]:
# TODO: METER TODO EN EL TRY EXCEPT
# TODO: add sleep
# Poner en una funcion

In [11]:
id = driver.find_element_by_xpath("//h3[contains(text(),\'SEGUNDA')]").get_attribute("id")
ActionChains(driver).move_to_element(driver.find_element_by_id(id)).perform()

<ipython-input-11-06805c9f29b9>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  id = driver.find_element_by_xpath("//h3[contains(text(),\'SEGUNDA')]").get_attribute("id")
<ipython-input-11-06805c9f29b9>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ActionChains(driver).move_to_element(driver.find_element_by_id(id)).perform()


In [19]:
def download_pdf(driver):
    """
    Dowloads the the PDF files.
    
    Params
    -------
    driver : selenium.webdriver.chrome.webdriver.WebDriver
        The web driver.
    """
    # Get all h4 titles after the second section title
    titles = driver.find_elements_by_xpath("//h3[contains(text(),\'SEGUNDA')]/following-sibling::h4")
    # Iterate through the titles
    for element in titles:
        # Get the directory name
        subdirectory = f"{directory}/{element.text}"
        # Create directory
        if not exists(subdirectory):
            makedirs(subdirectory)
        # Find the pdfs after the title
        pdfs = element.find_element_by_xpath("./following-sibling::ul").find_elements_by_xpath(".//li[@class = 'puntoPDF']/a")
        for pdf in pdfs:
    #         print(pdf.get_attribute('href'))
            URL = pdf.get_attribute('href')
            file_name = URL.split("/")[-1]
            logging.info(f"{subdirectory} ----- {URL}")
            # Using response in stead of clicking to control the directory the pdfs are downloaded into
            response = requests.get(URL)  
            with open(f"{subdirectory}/{file_name}", "wb") as output_file:
                output_file.write(response.content)
            time.sleep(2)

In [17]:
download_pdf(driver)

<ipython-input-16-6f19a194b0b3>:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  titles = driver.find_elements_by_xpath("//h3[contains(text(),\'SEGUNDA')]/following-sibling::h4")
C:\Users\Maria\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\Users\Maria\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
INFO:root:Boletin-20220105/CONVOCATORIAS DE JUNTAS ----- https://www.boe.es/borme/dias/2022/01/05/pdfs/BORME-C-2022-43.pdf


2


INFO:root:Boletin-20220105/FUSIONES Y ABSORCIONES DE EMPRESAS ----- https://www.boe.es/borme/dias/2022/01/05/pdfs/BORME-C-2022-44.pdf


In [21]:
def spider(driver):
    """
    Retrieves all the XML links.
    Params
    -------
    driver : selenium.webdriver.chrome.webdriver.WebDriver
        The web driver.

    Return
    -------
    links : list
        Matrix with columns for directory folder and link.
    """
    links = []
    # Get all h4 titles after the second section title
    titles = driver.find_elements_by_xpath("//h3[contains(text(),\'SEGUNDA')]/following-sibling::h4")
    # Iterate through the titles
    for element in titles:
        # Get the directory name
        subdirectory = f"{directory}/{element.text}"
        # Create directory
        if not exists(subdirectory):
            makedirs(subdirectory)
        # Find the pdfs after the title
        other_formats = element.find_element_by_xpath("./following-sibling::ul").find_elements_by_xpath(".//li[@class = 'puntoHTML']/a")
        for other_format in other_formats:
            links.append([element.text, other_format.get_attribute("href")])
    return links

In [22]:
links = spider(driver)

<ipython-input-21-8f3105cd3ede>:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  titles = driver.find_elements_by_xpath("//h3[contains(text(),\'SEGUNDA')]/following-sibling::h4")


In [23]:
def download_xml(driver, links):
    """
    Dowloads the the xml file in each link.
    
    Params
    -------
    driver : selenium.webdriver.chrome.webdriver.WebDriver
        The web driver.
    links : list
        Matrix with columns for directory folder and link.

    """
    for link in links:
        subdirectory = f"{directory}/{link[0]}"
        url = link[1]
        driver.get(url)
        URL = driver.find_element_by_xpath(".//li[@class = 'puntoXML']/a").get_attribute('href')
        # Get file name from query string
        file_name = URL.split("=")[-1]
        logging.info(f"{subdirectory} ----- {URL}")
        # Using response in stead of clicking to control the directory the pdfs are downloaded into
        response = requests.get(URL)  
        with open(f"{subdirectory}/{file_name}.xml", "wb") as output_file:
            output_file.write(response.content)
        time.sleep(2)

In [24]:
download_xml(driver, links)

<ipython-input-23-feefad750095>:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  URL = driver.find_element_by_xpath(".//li[@class = 'puntoXML']/a").get_attribute('href')
INFO:root:Boletin-20220105/CONVOCATORIAS DE JUNTAS ----- https://www.boe.es/diario_borme/xml.php?id=BORME-C-2022-43
INFO:root:Boletin-20220105/FUSIONES Y ABSORCIONES DE EMPRESAS ----- https://www.boe.es/diario_borme/xml.php?id=BORME-C-2022-44
